# Exploring Data (lesson 1)

In [53]:
#importing packages
from __future__ import print_function
import os
import re
import string
import tensorflow as tf
from random import randint
from pickle import load
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences
from numpy import array
from pickle import dump
from pickle import load
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
from numpy import array
from numpy import asarray
from numpy import zeros
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding

Starting of by downloading data and looking at the data, which we need to clean. We observe following
- Illustration texts
- The text file contain multiple stories, these should be split into multiple files in order for the model to know there is groupings (Extension: will be reviewed in lesson 5)
- The file contains a number of punctuations (,;.* etc.), which are irrelevant for the model
- Capital letters as headers

The input sequences are at first at 50 across sentences, chapters and stories. 

In [12]:
# creating function to load documents into memory
def load_doc(filename):
	file = open(filename, 'r', encoding="utf8") #loading an existing file
	text = file.read() #opening the file and assigning it to the variable text
	file.close() #close the file
	return text #output = text

In [13]:
#concatenate text files
path = 'Data'
 
files = os.listdir(path)

filenames = ['Data/22693-0.txt', 'Data/2892-0.txt']

with open('Data/adventures.txt', 'w', encoding="utf8") as outfile:
    for fname in filenames:
        with open(fname, encoding="utf8") as infile:
            outfile.write(infile.read())

In [14]:
# load document
in_filename = 'Data/adventures.txt'#specifying the filename of the data we wish to load
doc = load_doc(in_filename) #loading the file
print(doc[:10000]) #printing the first 200 characters of the loading document

﻿Just as a little child holds out its hands to catch the sunbeams, to
feel and to grasp what, so its eyes tell it, is actually there, so,
down through the ages, men have stretched out their hands in eager
endeavour to know their God. And because only through the human was
the divine knowable, the old peoples of the earth made gods of their
heroes and not unfrequently endowed these gods with as many of the
vices as of the virtues of their worshippers. As we read the myths of
the East and the West we find ever the same story. That portion of the
ancient Aryan race which poured from the central plain of Asia,
through the rocky defiles of what we now call "The Frontier," to
populate the fertile lowlands of India, had gods who must once have
been wholly heroic, but who came in time to be more degraded than the
most vicious of lustful criminals. And the Greeks, Latins, Teutons,
Celts, and Slavonians, who came of the same mighty Aryan stock, did
even as those with whom they owned a common anc

# Cleaning data (lesson 2)

In lesson 1, we observed several potential issues in the data, which we need to remove from the data. This is done below:

In [15]:
#removing illustration descriptions
doc = re.sub(r'\[[^)]*\]', '', doc) #using re.sub function and regular expressions 
#[ - an opening bracket 
#[^()]* - zero or more characters other than those defined, that is, any characters other than [ and ]
#\] - a closing bracket


In [16]:
#removing headers
doc = re.sub(r'[A-Z]{2,}','', doc) #replacing capital letters longer than 1 with nothing. 


In [17]:
#removing special characters, since one of the deliminators
#doc = doc.replace("'", '')

#deliminators 
#deliminators = c('THE FIR TREE', 'LITTLE TUK', 'THE UGLY DUCKLING', 'LITTLE IDA'S FLOWERS', 'THE STEADFAST TIN SOLDIER, LITTLE THUMBELINA, SUNSHINE STORIES', 'THE DARNING-NEEDLE', 'THE LITTLE MATCH GIRL', 'THE LOVING PAIR', 'THE LEAPING MATCH', 'THE HAPPY FAMILY, THE GREENIES, 'OLE-LUK-OIE', 'THE DREAM GOD', 'THE MONEY BOX', 'ELDER-TREE MOTHER', 'THE SNOW QUEEN', 'THE ROSES AND THE SPARROWS', 'THE OLD HOUSE', 'THE CONCEITED APPLE BRANCH' 

#splitting the file into strings consisting of one adventure
#doc = doc.split ()

In [18]:
# turn a doc into clean tokens
def clean_doc(doc): #making a function
	# split into tokens by white space
	tokens = doc.split()
	# remove punctuation from each token - https://www.geeksforgeeks.org/python-maketrans-translate-functions/
	table = str.maketrans('', '', string.punctuation) #Third argument specifies the wished deleted items
	tokens = [w.translate(table) for w in tokens] #translate applies the translation table applied on the looping through the tokens list
	# remove remaining tokens that are not alphabetic
	tokens = [word for word in tokens if word.isalpha()] #Replace word with word, if the word is alphabetic
	# make lower case
	tokens = [word.lower() for word in tokens]
	return tokens

In [19]:
tokens = clean_doc(doc) #using the clean function to cleaning our document
print(tokens[:200])
print('Total Tokens: %d' % len(tokens)) #printing the number of tokens
print('Unique Tokens: %d' % len(set(tokens))) #printing the number of unique tokens by grouping similair tokens

['as', 'a', 'little', 'child', 'holds', 'out', 'its', 'hands', 'to', 'catch', 'the', 'sunbeams', 'to', 'feel', 'and', 'to', 'grasp', 'what', 'so', 'its', 'eyes', 'tell', 'it', 'is', 'actually', 'there', 'so', 'down', 'through', 'the', 'ages', 'men', 'have', 'stretched', 'out', 'their', 'hands', 'in', 'eager', 'endeavour', 'to', 'know', 'their', 'god', 'and', 'because', 'only', 'through', 'the', 'human', 'was', 'the', 'divine', 'knowable', 'the', 'old', 'peoples', 'of', 'the', 'earth', 'made', 'gods', 'of', 'their', 'heroes', 'and', 'not', 'unfrequently', 'endowed', 'these', 'gods', 'with', 'as', 'many', 'of', 'the', 'vices', 'as', 'of', 'the', 'virtues', 'of', 'their', 'worshippers', 'as', 'we', 'read', 'the', 'myths', 'of', 'the', 'east', 'and', 'the', 'west', 'we', 'find', 'ever', 'the', 'same', 'story', 'that', 'portion', 'of', 'the', 'ancient', 'aryan', 'race', 'which', 'poured', 'from', 'the', 'central', 'plain', 'of', 'asia', 'through', 'the', 'rocky', 'defiles', 'of', 'what', 'w

In [20]:
# organize into sequences of tokens
length = 50 + 1 #defining the length of the sequence
sequences = list() #creating an empty list
for i in range(length, len(tokens)): #range takes two arguments: start point and end point
	# select sequence of tokens
	seq = tokens[i-length:i]
	# convert into a line
	line = ' '.join(seq) #-join returns a string in which the string elements of sequence have been joined by str separator
	# store
	sequences.append(line)
print('Total Sequences: %d' % len(sequences))

Total Sequences: 81379


In [21]:
# save tokens to file, one dialog per line
def save_doc(lines, filename): #creating a saving function
	data = '\n'.join(lines) #joining the lines seperated by 'enter'
	file = open(filename, 'w', encoding="utf8") #create empty new file
	file.write(data) #input the data in the empty file
	file.close() #close the file

In [22]:
# save sequences to file
out_filename = 'Data/fairytales_sequences.txt'
save_doc(sequences, out_filename)

# Training the model (Lesson 3)

In [23]:
# loading the cleaned data
in_filename = 'Data/fairytales_sequences.txt'
doc = load_doc(in_filename)

#split the text by lineshifts creating a list of 51 items long string
lines = doc.split('\n')

In [24]:
# integer encode sequences of words
tokenizer = Tokenizer() #assign the tokenizer function to a variable
tokenizer.fit_on_texts(lines) #function of keras finds all of the unique words in the data and assigns each a unique integer
sequences = tokenizer.texts_to_sequences(lines) #translating the input lines into integers
tokenizer.word_index #checking the dictionary of the transformed wordsb

{'the': 1,
 'and': 2,
 'of': 3,
 'to': 4,
 'a': 5,
 'he': 6,
 'that': 7,
 'in': 8,
 'was': 9,
 'his': 10,
 'it': 11,
 'for': 12,
 'with': 13,
 'said': 14,
 'her': 15,
 'is': 16,
 'as': 17,
 'on': 18,
 'not': 19,
 'had': 20,
 'she': 21,
 'they': 22,
 'i': 23,
 'but': 24,
 'at': 25,
 'from': 26,
 'him': 27,
 'be': 28,
 'you': 29,
 'were': 30,
 'all': 31,
 'there': 32,
 'my': 33,
 'by': 34,
 'when': 35,
 'one': 36,
 'fionn': 37,
 'their': 38,
 'then': 39,
 'would': 40,
 'so': 41,
 'this': 42,
 'king': 43,
 'have': 44,
 'who': 45,
 'them': 46,
 'will': 47,
 'no': 48,
 'or': 49,
 'an': 50,
 'which': 51,
 'could': 52,
 'me': 53,
 'great': 54,
 'into': 55,
 'did': 56,
 'man': 57,
 'came': 58,
 'are': 59,
 'out': 60,
 'what': 61,
 'we': 62,
 'do': 63,
 'more': 64,
 'been': 65,
 'if': 66,
 'your': 67,
 'time': 68,
 'men': 69,
 'mongan': 70,
 'went': 71,
 'than': 72,
 'like': 73,
 'come': 74,
 'mac': 75,
 'these': 76,
 'world': 77,
 'yet': 78,
 'love': 79,
 'other': 80,
 'up': 81,
 'heart': 82,


In [25]:
#calculating vocabulary size to estimate the size of the embedding layer
vocab_size = len(tokenizer.word_index) + 1 #since indexing of array are zero-offset, the index of the vocabulary must be one larger than the length
vocab_size

7874

In [26]:
# separate into input and output
sequences = array(sequences) #transforming the sequens of integers to arrays
X, y = sequences[:,:-1], sequences[:,-1] #defining X (input sequences) and y (output words)
y = to_categorical(y, num_classes=vocab_size) #to_categorical converts a class vector (integers) to binary class matrix
seq_length = X.shape[1] #gives you the dimension of the array, which we put to be 50
seq_length

50

We will use a two LSTM hidden layers with 100 memory cells each. More memory cells and a deeper network may achieve better results.

In [47]:
embed_dim = 50
# define model
model = Sequential() #assigning the sequential function to a model
model.add(Embedding(vocab_size, embed_dim, input_length=seq_length)) #defining embedding layer size
model.add(LSTM(100, return_sequences=True)) #adding layer of nodes
model.add(LSTM(100))  #adding layer of nodes
model.add(Dense(100, activation='relu')) #specifying the structure of the hidden layer, recu is an argument of a rectified linear unit. 
model.add(Dense(vocab_size, activation='softmax')) #using the softmax function to creating probabilities
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 50, 50)            393700    
_________________________________________________________________
lstm_5 (LSTM)                (None, 50, 100)           60400     
_________________________________________________________________
lstm_6 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_5 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_6 (Dense)              (None, 7874)              795274    
Total params: 1,339,874
Trainable params: 1,339,874
Non-trainable params: 0
_________________________________________________________________
None


In [28]:
# compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']) #the compile function configures the model for training specifying the categorical cross entropy loss
# fit model
#model.fit(X, y, batch_size=128, epochs=100) #training the model 

In [29]:
# save the model to file
model.save('model.h5')
# save the tokenizer
dump(tokenizer, open('tokenizer.pkl', 'wb'))

# Lesson 4

In [30]:
# load cleaned text sequences
doc = load_doc('Data/fairytales_sequences.txt')
lines = doc.split('\n')

In [31]:
#Specifying input sequences length to prompt the model
seq_length = len(lines[3].split()) - 1 #splitting the sequences into words, counting them -1
seq_length

50

In [32]:
#loading model
model= load_model('model.h5')
tokenizer = load(open('tokenizer.pkl', 'rb')) #r for read

In [33]:
#selecting a seed text
seed_text = lines[randint(0,len(lines))] #returns random integer between 0 and how many lines there is, and indexes this. 
print(seed_text + '\n') #prints the selected text
len(seed_text.split())

protect he continued see you killed said he lamentably am getting down now to fight the will not hurt said the stranger are the king asked am the son of fiachna knew then that the stranger could not be hurt will you give me if i deliver you from the asked



51

In [34]:
#translating the input text to integers
encoded = tokenizer.texts_to_sequences([seed_text])[0]
encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')

The model can predict the next word directly by calling model.predict_classes() that will return the index of the word with the highest probability

In [35]:
# predict probabilities for each word
yhat = model.predict_classes(encoded, verbose=1)

1/1 [==============================] - 1s 527ms/step


Looking up the index in the Tokenizers mapping to get the associated word

In [36]:
#making a loop to translate integer to word
out_word = ''
for word, index in tokenizer.word_index.items():
	if index == yhat:
		out_word = word
		break

out_word

'certainly'

The input sequences will get too long, in order to keep them to 50 items using the following function, which pads sequences to the same lengt

In [37]:
#Setting the max length to be 50 items by removing items from the beginnning of the sequence
encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')

In [38]:
#creating a function, which generates the predicted output
def generate_seq(model, tokenizer, seq_length, seed_text, n_words):
	result = list() #make an empty list
	in_text = seed_text
	# generate a fixed number of words
	for _ in range(n_words):
		# encode the text as integer
		encoded = tokenizer.texts_to_sequences([in_text])[0]
		# truncate sequences to a fixed length
		encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
		# predict probabilities for each word
		yhat = model.predict_classes(encoded, verbose=0)
		# map predicted word index to word
		out_word = ''
		for word, index in tokenizer.word_index.items():
			if index == yhat:
				out_word = word
				break
		# append to input
		in_text += ' ' + out_word
		result.append(out_word)
	return ' '.join(result)

In [39]:
# generate new text
generated = generate_seq(model, tokenizer, seq_length, seed_text, 100)
print(generated)

#saving generated text
out_filename = 'Data/first_output.txt'
save_doc(generated, out_filename)


certainly younger younger bees line ratio ratio agreeable agreeable agreeable inspires dizzy dizzy dizzy dizzy dizzy dizzy dizzy dizzy dizzy dizzy dizzy dizzy dizzy dizzy dizzy dizzy dizzy dew dew dizzy dew dew dew dew soundly soundly rage rage seventh seventh doorway duck duck duck feel feel heaven heaven gigantic gigantic gigantic line pluck lamentably lamentably lamentably lamentably twist ink ink ink ink ink ink ihr ihr besought besought pluck pluck wellpacked wellpacked wellpacked line ihr ihr line prepared protects ensued ensued loves loves loves curse curse dodge dodge dodge complained complained finnachy finnachy finnachy easier easier easier easier informant


# Lesson 5: Advancing the model


Pre-Trained Word Embedding. Extend the model to use pre-trained word2vec or GloVe vectors to see if it results in a better model.

In [55]:
# load the whole embedding into memory
embeddings_index = dict()
f = open('Data/glove.6B/glove.6B.100d.txt', encoding = 'utf8')
for line in f:
	values = line.split()
	word = values[0]
	coefs = asarray(values[1:], dtype='float32')
	embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 400000 word vectors.


In [57]:
# create a weight matrix for words in training docs
embedding_matrix = zeros((vocab_size, 100))
for word, i in tokenizer.word_index.items():
	embedding_vector = embeddings_index.get(word)
	if embedding_vector is not None:
		embedding_matrix[i] = embedding_vector

In [58]:
e = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=4, trainable=False)

In [ ]:
# define model
model = Sequential()
e = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=4, trainable=False)
model.add(e)
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
# summarize the model
print(model.summary())
# fit the model
model.fit(encoded, epochs=50, verbose=0)

Things to be consider to advance the model: 

- We could process the data so that the model only ever deals with self-contained sentences and pad or truncate the text to meet this requirement for each input sequence. You could explore this as an extension to this tutorial.
- MORE DATA!

